# Librerías

In [62]:
import pandas as pd
import numpy as np
import os
import folium 
from sklearn.impute import SimpleImputer

from scipy.stats import ks_2samp

from sklearn.feature_selection import VarianceThreshold

import seaborn as sns

from varclushi import VarClusHi
from scipy.stats import ks_2samp
from sklearn.preprocessing import KBinsDiscretizer

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score, accuracy_score, confusion_matrix

from sklearn.linear_model import LogisticRegression

from scikitplot.metrics import plot_roc_curve
from functools import reduce

#Estas son las bibliotecas utilizadas a lo largo del módulo, las cuales son útiles para realizar análisis de datos.

# Lectura/carga de datos

In [2]:
df = pd.read_csv('all_month.csv')

# Seleccionamos el conjunto de datos en formato CSV utilizando 
# la función pd.read_csv y lo asignamos a la variable df.

# Funciones


In [3]:
from funciones import *
# Cargamos las funciones 

# Pre analisis

In [4]:
# Con la intención de entender mejor el conjunto de datos, realizamos un 
# pequeño análisis de las celdas y columnas contenidas en el archivo.

In [5]:
df.shape
# Con esta funcion vemos la cantidad de filas y columnas respectivamente de tu DataFrame

(9399, 22)

In [6]:
df.head()
# Con esta funcion vemos las primeras 5 filas del DataFrame

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-02-06T09:24:52.450Z,33.527000,-116.779167,11.64,0.94,ml,9.0,240.0,0.05186,0.11,...,2023-02-06T09:28:26.675Z,"10km WSW of Anza, CA",earthquake,2.43,1.970,0.331,11.0,automatic,ci,ci
1,2023-02-06T09:04:16.590Z,19.193001,-155.392838,32.18,2.77,md,33.0,180.0,NaN,0.12,...,2023-02-06T09:07:27.210Z,"9 km E of Pāhala, Hawaii",earthquake,0.62,0.780,2.770,4.0,automatic,hv,hv
2,2023-02-06T08:58:45.507Z,61.110200,-151.162700,53.90,2.00,ml,NaN,NaN,NaN,0.75,...,2023-02-06T09:01:03.183Z,"4 km NNW of Tyonek, Alaska",earthquake,NaN,1.000,NaN,NaN,automatic,ak,ak
3,2023-02-06T08:52:37.335Z,37.449000,37.233400,10.00,4.80,mb,58.0,61.0,0.27700,0.78,...,2023-02-06T09:31:48.040Z,"7 km SW of Pazarcık, Turkey",earthquake,7.54,1.884,0.072,60.0,reviewed,us,us
4,2023-02-06T08:51:11.930Z,38.842667,-122.829666,2.48,0.66,md,11.0,115.0,0.01295,0.03,...,2023-02-06T09:15:15.395Z,"10km WNW of Cobb, CA",earthquake,0.33,0.990,0.240,12.0,automatic,nc,nc


In [7]:
df.describe()
# Obtener estadísticas descriptivas resumidas de un DataFrame, como la media, la mediana, 
# la desviación estándar, el mínimo y el máximo de cada columna numérica

,latitude,longitude,depth,mag,nst,gap,dmin,rms,horizontalError,depthError,magError,magNst
count,9399.000000,9399.000000,9399.000000,9399.000000,6735.000000,6734.000000,5350.000000,9399.000000,6232.000000,9399.000000,6667.000000,6724.000000
mean,41.267491,-116.017013,26.933842,1.633398,24.194061,118.276677,0.697541,0.308940,1.822996,2.115687,0.256163,16.267995
std,19.751939,68.529045,57.151409,1.262600,22.996614,63.351387,2.258458,0.292379,3.174955,10.765662,0.429285,23.914914
min,-64.428900,-179.994000,-3.410000,-1.120000,0.000000,11.000000,0.000000,0.000000,0.070000,0.000000,0.000000,0.000000
25%,33.928250,-152.681700,3.090000,0.860000,9.000000,69.000000,0.013690,0.090000,0.260000,0.400000,0.106000,6.000000
50%,38.831501,-122.817001,9.300000,1.400000,17.000000,103.000000,0.057880,0.180000,0.430000,0.700000,0.166000,10.000000
75%,58.250083,-116.764583,28.780000,2.100000,32.000000,157.000000,0.207850,0.510000,0.980000,1.500000,0.242312,18.000000
max,84.884100,179.983300,653.645000,7.800000,345.000000,352.540000,41.027000,4.090000,21.360000,946.400000,5.270000,502.000000


# Diccionario de datos 

    - latitude : float64 que representa la latitud del epicentro del evento sísmico.
    - longitude: float64 que representa la longitud del epicentro del evento sísmico.
    - depth    : float64 que representa la profundidad del evento sísmico en kilómetros.
    - mag      : float64 que representa la magnitud del evento sísmico en la escala de Richter.
    - magType  : objeto (string) que representa el tipo de escala de magnitud utilizada para el evento sísmico.
    - nst      : float64 que representa el número de estaciones sismográficas que registraron el evento sísmico.
    - gap      : float64 que representa el ángulo de separación máximo entre las estaciones sismográficas más                        cercanas que registraron el evento sísmico.
    - dmin     : float64 que representa la distancia en grados entre el epicentro del evento sísmico y la estación                    sismográfica más cercana que lo registró.
    - rms      : float64 que representa la amplitud de la señal registrada por las estaciones sismográficas para el                  evento sísmico.
    - net      : objeto (string) que representa la red sismográfica que registró el evento sísmico.
    - id       : objeto (string) que representa el identificador único del evento sísmico.
    - updated  : objeto (string) que representa la fecha y hora en que se actualizó por última vez el registro del                    evento sísmico.
    - place    : objeto (string) que representa el lugar donde ocurrió el evento sísmico.
    - type     : objeto (string) que representa el tipo de evento sísmico.
    - horizontalError: float64 que representa el error en la localización horizontal del epicentro del evento                              sísmico.
    - depthError: float64 que representa el error en la localización vertical (profundidad) del epicentro del evento                   sísmico.
    - magError : float64 que representa el error en la magnitud del evento sísmico.
    - magNst   : float64 que representa el número de estaciones sismográficas que se utilizaron para calcular la                      magnitud del evento sísmico.
    - status   : objeto (string) que representa el estado del registro del evento sísmico.
    - locationSource: objeto (string) que representa la fuente de datos utilizada para la localización del evento                         sísmico.
    - magSource: objeto (string) que representa la fuente de datos utilizada para calcular la magnitud del evento                    sísmico.

In [8]:
df.dtypes
# mostrar los tipos de datos de las columnas en un DataFrame. 
# en este caso en particular

# 'float64'se refiere a variables que contienen números con decimales, como 1.2, 3.14 o 100.0.
# 'object' se refiere a variables que contienen texto o cadenas de caracteres, como "hola", "mundo" o "terremoto".

time                object
latitude           float64
longitude          float64
depth              float64
mag                float64
magType             object
nst                float64
gap                float64
dmin               float64
rms                float64
net                 object
id                  object
updated             object
place               object
type                object
horizontalError    float64
depthError         float64
magError           float64
magNst             float64
status              object
locationSource      object
magSource           object
dtype: object

# Limpieza / tratamiento

In [9]:
df.head(5)

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-02-06T09:24:52.450Z,33.527000,-116.779167,11.64,0.94,ml,9.0,240.0,0.05186,0.11,...,2023-02-06T09:28:26.675Z,"10km WSW of Anza, CA",earthquake,2.43,1.970,0.331,11.0,automatic,ci,ci
1,2023-02-06T09:04:16.590Z,19.193001,-155.392838,32.18,2.77,md,33.0,180.0,NaN,0.12,...,2023-02-06T09:07:27.210Z,"9 km E of Pāhala, Hawaii",earthquake,0.62,0.780,2.770,4.0,automatic,hv,hv
2,2023-02-06T08:58:45.507Z,61.110200,-151.162700,53.90,2.00,ml,NaN,NaN,NaN,0.75,...,2023-02-06T09:01:03.183Z,"4 km NNW of Tyonek, Alaska",earthquake,NaN,1.000,NaN,NaN,automatic,ak,ak
3,2023-02-06T08:52:37.335Z,37.449000,37.233400,10.00,4.80,mb,58.0,61.0,0.27700,0.78,...,2023-02-06T09:31:48.040Z,"7 km SW of Pazarcık, Turkey",earthquake,7.54,1.884,0.072,60.0,reviewed,us,us
4,2023-02-06T08:51:11.930Z,38.842667,-122.829666,2.48,0.66,md,11.0,115.0,0.01295,0.03,...,2023-02-06T09:15:15.395Z,"10km WNW of Cobb, CA",earthquake,0.33,0.990,0.240,12.0,automatic,nc,nc


In [10]:
df.isnull()
# para verificar si hay valores nulos o faltantes en nuestro DataFrame

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,True,True,True,False,...,False,False,False,True,False,True,True,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,False,False,False,False,False,False,True,True,True,False,...,False,False,False,True,False,True,True,False,False,False
9395,False,False,False,False,False,False,True,True,True,False,...,False,False,False,True,False,True,True,False,False,False
9396,False,False,False,False,False,False,True,True,True,False,...,False,False,False,True,False,True,True,False,False,False
9397,False,False,False,False,False,False,True,True,True,False,...,False,False,False,True,False,True,True,False,False,False


In [11]:
df.isnull().sum()
# Ver cada columa y ver cuantos valores vacios hay en cada una
# vemos que tenemos muchos valores faltantes los cuales son:

# nst (2664): representa el número de estaciones sismográficas que registraron el evento sísmico

# gap (2665): representa el ángulo de separación máximo entre las estaciones sismográficas más 
#             cercanas que registraron el evento sísmico.

# dmin(4049): representa la distancia en grados entre el epicentro del evento sísmico y la estación 
#             sismográfica más cercana que lo registró.

# horizontalError(3167): representa el error en la localización horizontal del epicentro del evento sísmico.

# magError(2732): representa el error en la magnitud del evento sísmico.

# magNst(2675): representa el número de estaciones sismográficas que se utilizaron para calcular la 
#               magnitud del evento sísmico.

time                  0
latitude              0
longitude             0
depth                 0
mag                   0
magType               0
nst                2664
gap                2665
dmin               4049
rms                   0
net                   0
id                    0
updated               0
place                 0
type                  0
horizontalError    3167
depthError            0
magError           2732
magNst             2675
status                0
locationSource        0
magSource             0
dtype: int64

In [12]:
c_v_p = ['nst', 'gap', 'dmin', 'horizontalError', 'magError', 'magNst']
# c_v_p: Columnas con valores perdidos 



In [13]:
imputer = SimpleImputer(strategy='mean')
df[c_v_p ] = imputer.fit_transform(df[c_v_p ])
# Imputación de los valores faltantes con la media de cada columna

In [14]:
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-02-06T09:24:52.450Z,33.527000,-116.779167,11.64,0.94,ml,9.000000,240.000000,0.051860,0.11,...,2023-02-06T09:28:26.675Z,"10km WSW of Anza, CA",earthquake,2.430000,1.970,0.331000,11.000000,automatic,ci,ci
1,2023-02-06T09:04:16.590Z,19.193001,-155.392838,32.18,2.77,md,33.000000,180.000000,0.697541,0.12,...,2023-02-06T09:07:27.210Z,"9 km E of Pāhala, Hawaii",earthquake,0.620000,0.780,2.770000,4.000000,automatic,hv,hv
2,2023-02-06T08:58:45.507Z,61.110200,-151.162700,53.90,2.00,ml,24.194061,118.276677,0.697541,0.75,...,2023-02-06T09:01:03.183Z,"4 km NNW of Tyonek, Alaska",earthquake,1.822996,1.000,0.256163,16.267995,automatic,ak,ak
3,2023-02-06T08:52:37.335Z,37.449000,37.233400,10.00,4.80,mb,58.000000,61.000000,0.277000,0.78,...,2023-02-06T09:31:48.040Z,"7 km SW of Pazarcık, Turkey",earthquake,7.540000,1.884,0.072000,60.000000,reviewed,us,us
4,2023-02-06T08:51:11.930Z,38.842667,-122.829666,2.48,0.66,md,11.000000,115.000000,0.012950,0.03,...,2023-02-06T09:15:15.395Z,"10km WNW of Cobb, CA",earthquake,0.330000,0.990,0.240000,12.000000,automatic,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,2023-01-07T09:56:14.929Z,61.695000,-146.491500,36.60,1.10,ml,24.194061,118.276677,0.697541,0.41,...,2023-01-25T02:21:24.901Z,"36 km SSE of Nelchina, Alaska",earthquake,1.822996,0.600,0.256163,16.267995,reviewed,ak,ak
9395,2023-01-07T09:52:03.738Z,62.316500,-151.554000,90.60,1.20,ml,24.194061,118.276677,0.697541,0.38,...,2023-01-25T02:21:24.696Z,"37 km NNW of Skwentna, Alaska",earthquake,1.822996,0.500,0.256163,16.267995,reviewed,ak,ak
9396,2023-01-07T09:50:20.885Z,69.150700,-144.563700,15.70,1.60,ml,24.194061,118.276677,0.697541,0.23,...,2023-01-26T00:27:22.082Z,"115 km SSW of Kaktovik, Alaska",earthquake,1.822996,2.200,0.256163,16.267995,reviewed,ak,ak
9397,2023-01-07T09:42:17.484Z,67.472500,-166.586100,19.50,1.80,ml,24.194061,118.276677,0.697541,0.31,...,2023-01-26T00:27:21.910Z,"91 km WSW of Kivalina, Alaska",earthquake,1.822996,1.300,0.256163,16.267995,reviewed,ak,ak


In [15]:
df.isnull()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9395,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9396,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9397,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [16]:
df.isnull().sum()

time               0
latitude           0
longitude          0
depth              0
mag                0
magType            0
nst                0
gap                0
dmin               0
rms                0
net                0
id                 0
updated            0
place              0
type               0
horizontalError    0
depthError         0
magError           0
magNst             0
status             0
locationSource     0
magSource          0
dtype: int64

In [17]:
df['nst'] = np.floor(df['nst'])
df['magNst'] = np.floor(df['magNst'])

#Redondeo hacia abajo

In [18]:
df


,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,...,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-02-06T09:24:52.450Z,33.527000,-116.779167,11.64,0.94,ml,9.0,240.000000,0.051860,0.11,...,2023-02-06T09:28:26.675Z,"10km WSW of Anza, CA",earthquake,2.430000,1.970,0.331000,11.0,automatic,ci,ci
1,2023-02-06T09:04:16.590Z,19.193001,-155.392838,32.18,2.77,md,33.0,180.000000,0.697541,0.12,...,2023-02-06T09:07:27.210Z,"9 km E of Pāhala, Hawaii",earthquake,0.620000,0.780,2.770000,4.0,automatic,hv,hv
2,2023-02-06T08:58:45.507Z,61.110200,-151.162700,53.90,2.00,ml,24.0,118.276677,0.697541,0.75,...,2023-02-06T09:01:03.183Z,"4 km NNW of Tyonek, Alaska",earthquake,1.822996,1.000,0.256163,16.0,automatic,ak,ak
3,2023-02-06T08:52:37.335Z,37.449000,37.233400,10.00,4.80,mb,58.0,61.000000,0.277000,0.78,...,2023-02-06T09:31:48.040Z,"7 km SW of Pazarcık, Turkey",earthquake,7.540000,1.884,0.072000,60.0,reviewed,us,us
4,2023-02-06T08:51:11.930Z,38.842667,-122.829666,2.48,0.66,md,11.0,115.000000,0.012950,0.03,...,2023-02-06T09:15:15.395Z,"10km WNW of Cobb, CA",earthquake,0.330000,0.990,0.240000,12.0,automatic,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,2023-01-07T09:56:14.929Z,61.695000,-146.491500,36.60,1.10,ml,24.0,118.276677,0.697541,0.41,...,2023-01-25T02:21:24.901Z,"36 km SSE of Nelchina, Alaska",earthquake,1.822996,0.600,0.256163,16.0,reviewed,ak,ak
9395,2023-01-07T09:52:03.738Z,62.316500,-151.554000,90.60,1.20,ml,24.0,118.276677,0.697541,0.38,...,2023-01-25T02:21:24.696Z,"37 km NNW of Skwentna, Alaska",earthquake,1.822996,0.500,0.256163,16.0,reviewed,ak,ak
9396,2023-01-07T09:50:20.885Z,69.150700,-144.563700,15.70,1.60,ml,24.0,118.276677,0.697541,0.23,...,2023-01-26T00:27:22.082Z,"115 km SSW of Kaktovik, Alaska",earthquake,1.822996,2.200,0.256163,16.0,reviewed,ak,ak
9397,2023-01-07T09:42:17.484Z,67.472500,-166.586100,19.50,1.80,ml,24.0,118.276677,0.697541,0.31,...,2023-01-26T00:27:21.910Z,"91 km WSW of Kivalina, Alaska",earthquake,1.822996,1.300,0.256163,16.0,reviewed,ak,ak


In [19]:

df['time'] = pd.to_datetime(df['time'])
df['updated'] = pd.to_datetime(df['updated'])
# Convertir columnas de tiempo a formato de fecha y hora

In [20]:

df['time'] = df['time'].dt.strftime('%Y-%m-%d %H:%M:%S')
df['updated'] = df['updated'].dt.strftime('%Y-%m-%d %H:%M:%S')
# Dar formato deseado

In [21]:
df['time'] = pd.to_datetime(df['time'])
df['updated'] = pd.to_datetime(df['updated'])

In [22]:
pd.options.display.max_columns = None

In [23]:
df['gap'] = df['gap'].round(2)
df['horizontalError'] = df['horizontalError'].round(2)
df['dmin'] = df['dmin'].round(2)
df['depthError'] = df['depthError'].round(2)
df['magError'] = df['magError'].round(2)
# redondeamos a dos decimales esas columnas

In [24]:
cols = ['depth', 'mag', 'nst', 'gap', 'dmin', 'rms', 'horizontalError', 'depthError', 'magError', 'magNst']


for col in cols:
    if df[col].min() < 0:
        print(f"La columna '{col}' tiene valores negativos.")
# Verificar si hay valores negativos en cada columna

La columna 'depth' tiene valores negativos.
La columna 'mag' tiene valores negativos.


In [25]:
df['depth'] = df['depth'].abs()
df['mag'] = df['mag'].abs()

In [26]:
df
# en este punto concidero que el dataframe esta listo para comenzar con la ingeneria de variables

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource
0,2023-02-06 09:24:52,33.527000,-116.779167,11.64,0.94,ml,9.0,240.00,0.05,0.11,ci,ci40411544,2023-02-06 09:28:26,"10km WSW of Anza, CA",earthquake,2.43,1.97,0.33,11.0,automatic,ci,ci
1,2023-02-06 09:04:16,19.193001,-155.392838,32.18,2.77,md,33.0,180.00,0.70,0.12,hv,hv73320217,2023-02-06 09:07:27,"9 km E of Pāhala, Hawaii",earthquake,0.62,0.78,2.77,4.0,automatic,hv,hv
2,2023-02-06 08:58:45,61.110200,-151.162700,53.90,2.00,ml,24.0,118.28,0.70,0.75,ak,ak0231pbzu3m,2023-02-06 09:01:03,"4 km NNW of Tyonek, Alaska",earthquake,1.82,1.00,0.26,16.0,automatic,ak,ak
3,2023-02-06 08:52:37,37.449000,37.233400,10.00,4.80,mb,58.0,61.00,0.28,0.78,us,us6000jlpv,2023-02-06 09:31:48,"7 km SW of Pazarcık, Turkey",earthquake,7.54,1.88,0.07,60.0,reviewed,us,us
4,2023-02-06 08:51:11,38.842667,-122.829666,2.48,0.66,md,11.0,115.00,0.01,0.03,nc,nc73842531,2023-02-06 09:15:15,"10km WNW of Cobb, CA",earthquake,0.33,0.99,0.24,12.0,automatic,nc,nc
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,2023-01-07 09:56:14,61.695000,-146.491500,36.60,1.10,ml,24.0,118.28,0.70,0.41,ak,ak023bqg8ol,2023-01-25 02:21:24,"36 km SSE of Nelchina, Alaska",earthquake,1.82,0.60,0.26,16.0,reviewed,ak,ak
9395,2023-01-07 09:52:03,62.316500,-151.554000,90.60,1.20,ml,24.0,118.28,0.70,0.38,ak,ak023bqfhfp,2023-01-25 02:21:24,"37 km NNW of Skwentna, Alaska",earthquake,1.82,0.50,0.26,16.0,reviewed,ak,ak
9396,2023-01-07 09:50:20,69.150700,-144.563700,15.70,1.60,ml,24.0,118.28,0.70,0.23,ak,ak023bqf3bc,2023-01-26 00:27:22,"115 km SSW of Kaktovik, Alaska",earthquake,1.82,2.20,0.26,16.0,reviewed,ak,ak
9397,2023-01-07 09:42:17,67.472500,-166.586100,19.50,1.80,ml,24.0,118.28,0.70,0.31,ak,ak023bqddbr,2023-01-26 00:27:21,"91 km WSW of Kivalina, Alaska",earthquake,1.82,1.30,0.26,16.0,reviewed,ak,ak


# Ingenieria de variables 

In [27]:
um = ['depth','hrs']

In [28]:
df['hrs'] = df['time'].map( lambda x: x.strftime('%Y%m%d%H') ).astype(int)

In [29]:
df.head()

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,hrs
0,2023-02-06 09:24:52,33.527000,-116.779167,11.64,0.94,ml,9.0,240.00,0.05,0.11,ci,ci40411544,2023-02-06 09:28:26,"10km WSW of Anza, CA",earthquake,2.43,1.97,0.33,11.0,automatic,ci,ci,2023020609
1,2023-02-06 09:04:16,19.193001,-155.392838,32.18,2.77,md,33.0,180.00,0.70,0.12,hv,hv73320217,2023-02-06 09:07:27,"9 km E of Pāhala, Hawaii",earthquake,0.62,0.78,2.77,4.0,automatic,hv,hv,2023020609
2,2023-02-06 08:58:45,61.110200,-151.162700,53.90,2.00,ml,24.0,118.28,0.70,0.75,ak,ak0231pbzu3m,2023-02-06 09:01:03,"4 km NNW of Tyonek, Alaska",earthquake,1.82,1.00,0.26,16.0,automatic,ak,ak,2023020608
3,2023-02-06 08:52:37,37.449000,37.233400,10.00,4.80,mb,58.0,61.00,0.28,0.78,us,us6000jlpv,2023-02-06 09:31:48,"7 km SW of Pazarcık, Turkey",earthquake,7.54,1.88,0.07,60.0,reviewed,us,us,2023020608
4,2023-02-06 08:51:11,38.842667,-122.829666,2.48,0.66,md,11.0,115.00,0.01,0.03,nc,nc73842531,2023-02-06 09:15:15,"10km WNW of Cobb, CA",earthquake,0.33,0.99,0.24,12.0,automatic,nc,nc,2023020608


In [30]:
cat_hrs = df[['hrs']].drop_duplicates().reset_index(drop=True)

In [31]:
cat_hrs['hrs'] = sorted( cat_hrs['hrs'].astype(int) )

In [32]:
cat_hrs['ancla'] = cat_hrs.index + 1

In [33]:
cat_hrs

,hrs,ancla
0,2023010709,1
1,2023010710,2
2,2023010711,3
3,2023010712,4
4,2023010713,5
...,...,...
716,2023020605,717
717,2023020606,718
718,2023020607,719
719,2023020608,720


# cónteo de sísmos

In [34]:
n_sismos_hr = df[ um + ['id'] ].groupby(um).count().reset_index().rename(columns={'id':'n_sismos'})

In [35]:
df = df.merge(n_sismos_hr, on=um)

In [36]:
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,hrs,n_sismos
0,2023-02-06 09:24:52,33.527000,-116.779167,11.64,0.94,ml,9.0,240.00,0.05,0.11,ci,ci40411544,2023-02-06 09:28:26,"10km WSW of Anza, CA",earthquake,2.43,1.97,0.33,11.0,automatic,ci,ci,2023020609,1
1,2023-02-06 09:04:16,19.193001,-155.392838,32.18,2.77,md,33.0,180.00,0.70,0.12,hv,hv73320217,2023-02-06 09:07:27,"9 km E of Pāhala, Hawaii",earthquake,0.62,0.78,2.77,4.0,automatic,hv,hv,2023020609,1
2,2023-02-06 08:58:45,61.110200,-151.162700,53.90,2.00,ml,24.0,118.28,0.70,0.75,ak,ak0231pbzu3m,2023-02-06 09:01:03,"4 km NNW of Tyonek, Alaska",earthquake,1.82,1.00,0.26,16.0,automatic,ak,ak,2023020608,1
3,2023-02-06 08:52:37,37.449000,37.233400,10.00,4.80,mb,58.0,61.00,0.28,0.78,us,us6000jlpv,2023-02-06 09:31:48,"7 km SW of Pazarcık, Turkey",earthquake,7.54,1.88,0.07,60.0,reviewed,us,us,2023020608,1
4,2023-02-06 08:51:11,38.842667,-122.829666,2.48,0.66,md,11.0,115.00,0.01,0.03,nc,nc73842531,2023-02-06 09:15:15,"10km WNW of Cobb, CA",earthquake,0.33,0.99,0.24,12.0,automatic,nc,nc,2023020608,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,2023-01-07 09:56:14,61.695000,-146.491500,36.60,1.10,ml,24.0,118.28,0.70,0.41,ak,ak023bqg8ol,2023-01-25 02:21:24,"36 km SSE of Nelchina, Alaska",earthquake,1.82,0.60,0.26,16.0,reviewed,ak,ak,2023010709,1
9395,2023-01-07 09:52:03,62.316500,-151.554000,90.60,1.20,ml,24.0,118.28,0.70,0.38,ak,ak023bqfhfp,2023-01-25 02:21:24,"37 km NNW of Skwentna, Alaska",earthquake,1.82,0.50,0.26,16.0,reviewed,ak,ak,2023010709,1
9396,2023-01-07 09:50:20,69.150700,-144.563700,15.70,1.60,ml,24.0,118.28,0.70,0.23,ak,ak023bqf3bc,2023-01-26 00:27:22,"115 km SSW of Kaktovik, Alaska",earthquake,1.82,2.20,0.26,16.0,reviewed,ak,ak,2023010709,1
9397,2023-01-07 09:42:17,67.472500,-166.586100,19.50,1.80,ml,24.0,118.28,0.70,0.31,ak,ak023bqddbr,2023-01-26 00:27:21,"91 km WSW of Kivalina, Alaska",earthquake,1.82,1.30,0.26,16.0,reviewed,ak,ak,2023010709,1


In [37]:
df['s_tipo_profu'] = np.where(df['depth'] <= 70, 'superficial',
                    np.where((df['depth'] >= 71) & (df['depth'] <= 299), 'intermedio',
                    np.where(df['depth'] >= 300, 'profundo', 'N/A')))
# creamos la columnas tipo_profu: tipo de profundidad 

In [38]:
df['s_region'] = df['place'].str.split('of', expand=True)[1].str.strip()
# generamos la columna region

In [39]:
df['s_danio'] = np.where(df['mag'] < 3.5, 'sin danios',
                    np.where((df['mag'] >= 3.5) & (df['mag'] <= 5.4), 'danios menores',
                    np.where((df['mag'] >= 5.5) & (df['mag'] <= 6), 'danios ligeros',
                    np.where((df['mag'] >= 6.1) & (df['mag'] <= 6.9), 'danios severos',
                    np.where((df['mag'] >= 7.0) & (df['mag'] <= 7.9), 'danios graves', 
                    np.where(df['mag'] >= 8.0, 'danios totales', ''))))))

# agragamos la columa de danio, la cual significa: cantidad de daño.

In [69]:
df['prec_gap'] = np.where(df['gap'] <= 90, 'muy preciso',
                np.where((df['gap'] >= 91) & (df['gap'] <= 119), 'regu preciso',
                np.where(df['gap'] >= 120, 'poco preciso', 'otro valor')))

#generamos la oclumna prec_gap: precision por gpa (angulo de separacion entre estaciones)

In [70]:
df

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,hrs,n_sismos,s_tipo_profu,s_region,s_danio,prec_gap
0,2023-02-06 09:24:52,33.527000,-116.779167,11.64,0.94,ml,9.0,240.00,0.05,0.11,ci,ci40411544,2023-02-06 09:28:26,"10km WSW of Anza, CA",earthquake,2.43,1.97,0.33,11.0,automatic,ci,ci,2023020609,1,superficial,"Anza, CA",sin danios,poco preciso
1,2023-02-06 09:04:16,19.193001,-155.392838,32.18,2.77,md,33.0,180.00,0.70,0.12,hv,hv73320217,2023-02-06 09:07:27,"9 km E of Pāhala, Hawaii",earthquake,0.62,0.78,2.77,4.0,automatic,hv,hv,2023020609,1,superficial,"Pāhala, Hawaii",sin danios,poco preciso
2,2023-02-06 08:58:45,61.110200,-151.162700,53.90,2.00,ml,24.0,118.28,0.70,0.75,ak,ak0231pbzu3m,2023-02-06 09:01:03,"4 km NNW of Tyonek, Alaska",earthquake,1.82,1.00,0.26,16.0,automatic,ak,ak,2023020608,1,superficial,"Tyonek, Alaska",sin danios,regu preciso
3,2023-02-06 08:52:37,37.449000,37.233400,10.00,4.80,mb,58.0,61.00,0.28,0.78,us,us6000jlpv,2023-02-06 09:31:48,"7 km SW of Pazarcık, Turkey",earthquake,7.54,1.88,0.07,60.0,reviewed,us,us,2023020608,1,superficial,"Pazarcık, Turkey",danios menores,muy preciso
4,2023-02-06 08:51:11,38.842667,-122.829666,2.48,0.66,md,11.0,115.00,0.01,0.03,nc,nc73842531,2023-02-06 09:15:15,"10km WNW of Cobb, CA",earthquake,0.33,0.99,0.24,12.0,automatic,nc,nc,2023020608,1,superficial,"Cobb, CA",sin danios,regu preciso
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,2023-01-07 09:56:14,61.695000,-146.491500,36.60,1.10,ml,24.0,118.28,0.70,0.41,ak,ak023bqg8ol,2023-01-25 02:21:24,"36 km SSE of Nelchina, Alaska",earthquake,1.82,0.60,0.26,16.0,reviewed,ak,ak,2023010709,1,superficial,"Nelchina, Alaska",sin danios,regu preciso
9395,2023-01-07 09:52:03,62.316500,-151.554000,90.60,1.20,ml,24.0,118.28,0.70,0.38,ak,ak023bqfhfp,2023-01-25 02:21:24,"37 km NNW of Skwentna, Alaska",earthquake,1.82,0.50,0.26,16.0,reviewed,ak,ak,2023010709,1,intermedio,"Skwentna, Alaska",sin danios,regu preciso
9396,2023-01-07 09:50:20,69.150700,-144.563700,15.70,1.60,ml,24.0,118.28,0.70,0.23,ak,ak023bqf3bc,2023-01-26 00:27:22,"115 km SSW of Kaktovik, Alaska",earthquake,1.82,2.20,0.26,16.0,reviewed,ak,ak,2023010709,1,superficial,"Kaktovik, Alaska",sin danios,regu preciso
9397,2023-01-07 09:42:17,67.472500,-166.586100,19.50,1.80,ml,24.0,118.28,0.70,0.31,ak,ak023bqddbr,2023-01-26 00:27:21,"91 km WSW of Kivalina, Alaska",earthquake,1.82,1.30,0.26,16.0,reviewed,ak,ak,2023010709,1,superficial,"Kivalina, Alaska",sin danios,regu preciso


In [68]:
df.to_excel('datos.xlsx', index=False)

# Inntegración

In [41]:
X_num = df.merge( cat_hrs , on='hrs' , how='inner' ).reset_index(drop=True)

In [42]:
X_num

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,hrs,n_sismos,s_tipo_profu,s_region,s_danio,ancla
0,2023-02-06 09:24:52,33.527000,-116.779167,11.64,0.94,ml,9.0,240.00,0.05,0.11,ci,ci40411544,2023-02-06 09:28:26,"10km WSW of Anza, CA",earthquake,2.43,1.97,0.33,11.0,automatic,ci,ci,2023020609,1,superficial,"Anza, CA",sin danios,721
1,2023-02-06 09:04:16,19.193001,-155.392838,32.18,2.77,md,33.0,180.00,0.70,0.12,hv,hv73320217,2023-02-06 09:07:27,"9 km E of Pāhala, Hawaii",earthquake,0.62,0.78,2.77,4.0,automatic,hv,hv,2023020609,1,superficial,"Pāhala, Hawaii",sin danios,721
2,2023-02-06 08:58:45,61.110200,-151.162700,53.90,2.00,ml,24.0,118.28,0.70,0.75,ak,ak0231pbzu3m,2023-02-06 09:01:03,"4 km NNW of Tyonek, Alaska",earthquake,1.82,1.00,0.26,16.0,automatic,ak,ak,2023020608,1,superficial,"Tyonek, Alaska",sin danios,720
3,2023-02-06 08:52:37,37.449000,37.233400,10.00,4.80,mb,58.0,61.00,0.28,0.78,us,us6000jlpv,2023-02-06 09:31:48,"7 km SW of Pazarcık, Turkey",earthquake,7.54,1.88,0.07,60.0,reviewed,us,us,2023020608,1,superficial,"Pazarcık, Turkey",danios menores,720
4,2023-02-06 08:51:11,38.842667,-122.829666,2.48,0.66,md,11.0,115.00,0.01,0.03,nc,nc73842531,2023-02-06 09:15:15,"10km WNW of Cobb, CA",earthquake,0.33,0.99,0.24,12.0,automatic,nc,nc,2023020608,1,superficial,"Cobb, CA",sin danios,720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,2023-01-07 09:56:14,61.695000,-146.491500,36.60,1.10,ml,24.0,118.28,0.70,0.41,ak,ak023bqg8ol,2023-01-25 02:21:24,"36 km SSE of Nelchina, Alaska",earthquake,1.82,0.60,0.26,16.0,reviewed,ak,ak,2023010709,1,superficial,"Nelchina, Alaska",sin danios,1
9395,2023-01-07 09:52:03,62.316500,-151.554000,90.60,1.20,ml,24.0,118.28,0.70,0.38,ak,ak023bqfhfp,2023-01-25 02:21:24,"37 km NNW of Skwentna, Alaska",earthquake,1.82,0.50,0.26,16.0,reviewed,ak,ak,2023010709,1,intermedio,"Skwentna, Alaska",sin danios,1
9396,2023-01-07 09:50:20,69.150700,-144.563700,15.70,1.60,ml,24.0,118.28,0.70,0.23,ak,ak023bqf3bc,2023-01-26 00:27:22,"115 km SSW of Kaktovik, Alaska",earthquake,1.82,2.20,0.26,16.0,reviewed,ak,ak,2023010709,1,superficial,"Kaktovik, Alaska",sin danios,1
9397,2023-01-07 09:42:17,67.472500,-166.586100,19.50,1.80,ml,24.0,118.28,0.70,0.31,ak,ak023bqddbr,2023-01-26 00:27:21,"91 km WSW of Kivalina, Alaska",earthquake,1.82,1.30,0.26,16.0,reviewed,ak,ak,2023010709,1,superficial,"Kivalina, Alaska",sin danios,1


In [43]:
X_num.drop('hrs',axis=1,inplace=True)

In [44]:
X_num

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,n_sismos,s_tipo_profu,s_region,s_danio,ancla
0,2023-02-06 09:24:52,33.527000,-116.779167,11.64,0.94,ml,9.0,240.00,0.05,0.11,ci,ci40411544,2023-02-06 09:28:26,"10km WSW of Anza, CA",earthquake,2.43,1.97,0.33,11.0,automatic,ci,ci,1,superficial,"Anza, CA",sin danios,721
1,2023-02-06 09:04:16,19.193001,-155.392838,32.18,2.77,md,33.0,180.00,0.70,0.12,hv,hv73320217,2023-02-06 09:07:27,"9 km E of Pāhala, Hawaii",earthquake,0.62,0.78,2.77,4.0,automatic,hv,hv,1,superficial,"Pāhala, Hawaii",sin danios,721
2,2023-02-06 08:58:45,61.110200,-151.162700,53.90,2.00,ml,24.0,118.28,0.70,0.75,ak,ak0231pbzu3m,2023-02-06 09:01:03,"4 km NNW of Tyonek, Alaska",earthquake,1.82,1.00,0.26,16.0,automatic,ak,ak,1,superficial,"Tyonek, Alaska",sin danios,720
3,2023-02-06 08:52:37,37.449000,37.233400,10.00,4.80,mb,58.0,61.00,0.28,0.78,us,us6000jlpv,2023-02-06 09:31:48,"7 km SW of Pazarcık, Turkey",earthquake,7.54,1.88,0.07,60.0,reviewed,us,us,1,superficial,"Pazarcık, Turkey",danios menores,720
4,2023-02-06 08:51:11,38.842667,-122.829666,2.48,0.66,md,11.0,115.00,0.01,0.03,nc,nc73842531,2023-02-06 09:15:15,"10km WNW of Cobb, CA",earthquake,0.33,0.99,0.24,12.0,automatic,nc,nc,1,superficial,"Cobb, CA",sin danios,720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,2023-01-07 09:56:14,61.695000,-146.491500,36.60,1.10,ml,24.0,118.28,0.70,0.41,ak,ak023bqg8ol,2023-01-25 02:21:24,"36 km SSE of Nelchina, Alaska",earthquake,1.82,0.60,0.26,16.0,reviewed,ak,ak,1,superficial,"Nelchina, Alaska",sin danios,1
9395,2023-01-07 09:52:03,62.316500,-151.554000,90.60,1.20,ml,24.0,118.28,0.70,0.38,ak,ak023bqfhfp,2023-01-25 02:21:24,"37 km NNW of Skwentna, Alaska",earthquake,1.82,0.50,0.26,16.0,reviewed,ak,ak,1,intermedio,"Skwentna, Alaska",sin danios,1
9396,2023-01-07 09:50:20,69.150700,-144.563700,15.70,1.60,ml,24.0,118.28,0.70,0.23,ak,ak023bqf3bc,2023-01-26 00:27:22,"115 km SSW of Kaktovik, Alaska",earthquake,1.82,2.20,0.26,16.0,reviewed,ak,ak,1,superficial,"Kaktovik, Alaska",sin danios,1
9397,2023-01-07 09:42:17,67.472500,-166.586100,19.50,1.80,ml,24.0,118.28,0.70,0.31,ak,ak023bqddbr,2023-01-26 00:27:21,"91 km WSW of Kivalina, Alaska",earthquake,1.82,1.30,0.26,16.0,reviewed,ak,ak,1,superficial,"Kivalina, Alaska",sin danios,1


# Variable ojetivo

In [45]:
X_num.rename( columns = { 'n_sismos': 'y' } , inplace=True )

In [46]:
X_num

,time,latitude,longitude,depth,mag,magType,nst,gap,dmin,rms,net,id,updated,place,type,horizontalError,depthError,magError,magNst,status,locationSource,magSource,y,s_tipo_profu,s_region,s_danio,ancla
0,2023-02-06 09:24:52,33.527000,-116.779167,11.64,0.94,ml,9.0,240.00,0.05,0.11,ci,ci40411544,2023-02-06 09:28:26,"10km WSW of Anza, CA",earthquake,2.43,1.97,0.33,11.0,automatic,ci,ci,1,superficial,"Anza, CA",sin danios,721
1,2023-02-06 09:04:16,19.193001,-155.392838,32.18,2.77,md,33.0,180.00,0.70,0.12,hv,hv73320217,2023-02-06 09:07:27,"9 km E of Pāhala, Hawaii",earthquake,0.62,0.78,2.77,4.0,automatic,hv,hv,1,superficial,"Pāhala, Hawaii",sin danios,721
2,2023-02-06 08:58:45,61.110200,-151.162700,53.90,2.00,ml,24.0,118.28,0.70,0.75,ak,ak0231pbzu3m,2023-02-06 09:01:03,"4 km NNW of Tyonek, Alaska",earthquake,1.82,1.00,0.26,16.0,automatic,ak,ak,1,superficial,"Tyonek, Alaska",sin danios,720
3,2023-02-06 08:52:37,37.449000,37.233400,10.00,4.80,mb,58.0,61.00,0.28,0.78,us,us6000jlpv,2023-02-06 09:31:48,"7 km SW of Pazarcık, Turkey",earthquake,7.54,1.88,0.07,60.0,reviewed,us,us,1,superficial,"Pazarcık, Turkey",danios menores,720
4,2023-02-06 08:51:11,38.842667,-122.829666,2.48,0.66,md,11.0,115.00,0.01,0.03,nc,nc73842531,2023-02-06 09:15:15,"10km WNW of Cobb, CA",earthquake,0.33,0.99,0.24,12.0,automatic,nc,nc,1,superficial,"Cobb, CA",sin danios,720
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9394,2023-01-07 09:56:14,61.695000,-146.491500,36.60,1.10,ml,24.0,118.28,0.70,0.41,ak,ak023bqg8ol,2023-01-25 02:21:24,"36 km SSE of Nelchina, Alaska",earthquake,1.82,0.60,0.26,16.0,reviewed,ak,ak,1,superficial,"Nelchina, Alaska",sin danios,1
9395,2023-01-07 09:52:03,62.316500,-151.554000,90.60,1.20,ml,24.0,118.28,0.70,0.38,ak,ak023bqfhfp,2023-01-25 02:21:24,"37 km NNW of Skwentna, Alaska",earthquake,1.82,0.50,0.26,16.0,reviewed,ak,ak,1,intermedio,"Skwentna, Alaska",sin danios,1
9396,2023-01-07 09:50:20,69.150700,-144.563700,15.70,1.60,ml,24.0,118.28,0.70,0.23,ak,ak023bqf3bc,2023-01-26 00:27:22,"115 km SSW of Kaktovik, Alaska",earthquake,1.82,2.20,0.26,16.0,reviewed,ak,ak,1,superficial,"Kaktovik, Alaska",sin danios,1
9397,2023-01-07 09:42:17,67.472500,-166.586100,19.50,1.80,ml,24.0,118.28,0.70,0.31,ak,ak023bqddbr,2023-01-26 00:27:21,"91 km WSW of Kivalina, Alaska",earthquake,1.82,1.30,0.26,16.0,reviewed,ak,ak,1,superficial,"Kivalina, Alaska",sin danios,1


# Definición de variables de matriz predictora

In [47]:
var = X_num.filter(like='s_').columns.to_list()

In [48]:
var

['s_tipo_profu', 's_region', 's_danio']

# Ventanas de tiempo 

## ingreso de ventanas de tiempo 

In [49]:
vdes = 1
vobs = 12

hr_i = cat_hrs['ancla'].min()
hr_f = cat_hrs['ancla'].max()

print(hr_i, hr_f)

ancla_i = hr_i + vobs - 1
ancla_f = hr_f - vdes

print(ancla_i, ancla_f)

1 721
12 720


In [50]:
def ventanas( df_aux, variable, ancla ):
    aux = df_aux.loc[ (df_aux.ancla >=  (ancla-vobs+1) ) & (df_aux.ancla <= ancla)].reset_index(drop=True)
    aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
    aux.columns = [ f's_{a}_{b}_{vobs}' for a,b in aux.columns ]
    aux.insert(0,'ancla',ancla)
    
    return aux.reset_index()

# Matriz de predictoras

In [51]:
X = pd.concat( map( lambda ancla:  ventanas( X_num , var , ancla) , range( ancla_i , ancla_f+1 )   ) , ignore_index=True  )

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','m

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','m

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','m

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','m

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ip

...

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','m

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','m

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','m

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','m

/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: pivot_table dropped a column because it failed to aggregate. This behavior is deprecated and will raise in a future version of pandas. Select only the columns that can be aggregated.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.max is deprecated. In a future version, a TypeError will be raised. Before calling .max, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','mean','max']  )
/tmp/ipykernel_7710/2425431411.py:3: FutureWarning: Dropping invalid columns in DataFrameGroupBy.min is deprecated. In a future version, a TypeError will be raised. Before calling .min, select only columns which should be valid for the function.
  aux = aux.pivot_table( index = ['depth'] , values=variable, aggfunc = ['min','m

In [52]:
X

,depth,ancla,s_min_s_danio_12,s_min_s_tipo_profu_12,s_max_s_danio_12,s_max_s_tipo_profu_12,s_min_s_region_12,s_max_s_region_12
0,0.000,12,sin danios,superficial,sin danios,superficial,NaN,NaN
1,1.080,12,sin danios,superficial,sin danios,superficial,NaN,NaN
2,1.350,12,sin danios,superficial,sin danios,superficial,NaN,NaN
3,1.460,12,sin danios,superficial,sin danios,superficial,NaN,NaN
4,1.510,12,sin danios,superficial,sin danios,superficial,NaN,NaN
...,...,...,...,...,...,...,...,...
101988,108.300,720,sin danios,intermedio,sin danios,intermedio,NaN,NaN
101989,126.894,720,danios menores,intermedio,danios menores,intermedio,NaN,NaN
101990,144.600,720,sin danios,intermedio,sin danios,intermedio,NaN,NaN
101991,343.249,720,danios menores,profundo,danios menores,profundo,NaN,NaN


# Vector solucion

In [53]:
def variable_obj(df,ancla):
    aux = df[['depth','ancla','y']].loc[ df.ancla == (ancla+vdes) ].reset_index(drop=True)
    return aux 

In [54]:
y = pd.concat( map( lambda ancla: variable_obj( X_num , ancla ), range(ancla_i, ancla_f+1) ) , ignore_index=True  )

In [55]:
y.head()

,depth,ancla,y
0,47.4,13,1
1,10.4,13,1
2,14.3,13,1
3,19.7,13,1
4,63.6,13,1


# TAD

In [56]:
tad = X.merge(y , on = ['depth','ancla'], how='inner').reset_index(drop=True)

In [57]:
tad

,depth,ancla,s_min_s_danio_12,s_min_s_tipo_profu_12,s_max_s_danio_12,s_max_s_tipo_profu_12,s_min_s_region_12,s_max_s_region_12,y
0,6.70,13,sin danios,superficial,sin danios,superficial,NaN,NaN,1
1,10.40,13,sin danios,superficial,sin danios,superficial,NaN,NaN,1
2,11.86,13,sin danios,superficial,sin danios,superficial,NaN,NaN,1
3,12.75,13,sin danios,superficial,sin danios,superficial,NaN,NaN,1
4,13.70,13,sin danios,superficial,sin danios,superficial,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...
9208,2.48,720,sin danios,superficial,sin danios,superficial,NaN,NaN,1
9209,10.00,720,danios ligeros,superficial,danios menores,superficial,NaN,NaN,1
9210,11.83,720,sin danios,superficial,sin danios,superficial,NaN,NaN,1
9211,25.80,720,sin danios,superficial,sin danios,superficial,NaN,NaN,1
